In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

from scipy import sparse

%matplotlib notebook

In [3]:
book=pd.read_csv('/Users/shraddhalipane/Desktop/MyProject/MyBooks/MyBooks/Books.csv', sep=',',encoding="ISO-8859-1",index_col= None, low_memory= False , dtype={'book_author': str})
rating=pd.read_csv('/Users/shraddhalipane/Desktop/MyProject/MyBooks/MyBooks/Ratings.csv')
user=pd.read_csv('/Users/shraddhalipane/Desktop/MyProject/Users.csv')

In [4]:
print(book.shape)
print(rating.shape)
print(user.shape)

(271360, 8)
(1048575, 3)
(278858, 3)


In [5]:
book.head()

,isbn,book_title,book_author,publication_year,Publisher,image_url_s,image_url_m,image_url_l
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


# Data Preprocessing

In [6]:
book.isnull().sum()

isbn                0
book_title          0
book_author         1
publication_year    0
Publisher           2
image_url_s         0
image_url_m         0
image_url_l         3
dtype: int64

In [7]:
missing_publishers = book[book['Publisher'].isnull()]
df_cleaned = book.dropna(subset=['Publisher'])
df_filled = book.fillna({'Publisher': 'Unknown'})
book['Publisher'].fillna(book['book_author'], inplace=True)
book.isnull().sum()

isbn                0
book_title          0
book_author         1
publication_year    0
Publisher           0
image_url_s         0
image_url_m         0
image_url_l         3
dtype: int64

In [8]:
missing_author = book[book['book_author'].isnull()]
ds_cleaned = book.dropna(subset=['book_author'])
df_filled_with= book.fillna({'book_author': 'Unknown'})
book['book_author'].fillna(book['Publisher'], inplace=True)

book.isnull().sum()

isbn                0
book_title          0
book_author         0
publication_year    0
Publisher           0
image_url_s         0
image_url_m         0
image_url_l         3
dtype: int64

In [9]:
book.dropna(inplace=True)
book.isnull().sum()

isbn                0
book_title          0
book_author         0
publication_year    0
Publisher           0
image_url_s         0
image_url_m         0
image_url_l         0
dtype: int64

In [10]:
rating.isnull().sum()

user_id        0
isbn           0
book_rating    0
dtype: int64

In [11]:
user.isnull().sum()

user_id          0
location         0
age         110762
dtype: int64

In [12]:
user.count()

user_id     278858
location    278858
age         168096
dtype: int64

In [13]:
user= pd.DataFrame(user)
user=user.fillna(0)
user.isnull().sum() 

user_id     0
location    0
age         0
dtype: int64

In [14]:
user.count()

user_id     278858
location    278858
age         278858
dtype: int64

In [15]:
book.to_csv('book.csv',index=False)

In [16]:

rating.to_csv('rating.csv',index=False)

In [17]:
book.publication_year.value_counts(dropna=False).sort_index().plot(kind='barh',figsize=(15,16))
plt.show()

<IPython.core.display.Javascript object>

In [18]:
combined_book_rating=pd.merge(book,rating,on='isbn')
columns= ['publication_year','Publisher','book_author','image_url_s','image_url_m','image_url_l']
combined_book_rating = combined_book_rating.drop(columns, axis=1)
combined_book_rating.head()

,isbn,book_title,user_id,book_rating
0,195153448,Classical Mythology,2,0
1,2005018,Clara Callan,8,5
2,2005018,Clara Callan,11400,0
3,2005018,Clara Callan,11676,8
4,2005018,Clara Callan,41385,0


In [19]:
combined_book_rating.groupby('book_title')['book_rating'].mean().sort_values(ascending=False).head()

book_title
The Burnt Orange Sunrise: A Berger &amp; Mitry Mystery                                                              10.0
The BetweenTime                                                                                                     10.0
The Best of Gourmet 2002: Featuring the Flavors of Paris                                                            10.0
The Best of Mennonite Fellowship Meals: More Tha 900 Favorite Recipes to Share With Friends at Home or at Church    10.0
WHISPERS: The Voices of Paranoia                                                                                    10.0
Name: book_rating, dtype: float64

In [20]:
combined_book_rating.groupby('book_title')['book_rating'].count().sort_values(ascending=False).head()

book_title
Wild Animus                   2264
The Lovely Bones: A Novel     1164
The Da Vinci Code              828
A Painted House                766
The Nanny Diaries: A Novel     759
Name: book_rating, dtype: int64

In [21]:
 #we will calculate the average rating using mean function

#avg_rating_df =combined_book_rating.groupby('book_title').mean()['book_rating'].reset_index()
#avg_rating_df.rename(columns={'book_rating':'avg_rating'},inplace=True)
#avg_rating_df.head()

In [22]:
#combined book data with rating data

combine_book_rating = pd.merge(rating, book, on='isbn',how='inner')
columns= ['publication_year','Publisher','book_author','image_url_s','image_url_m','image_url_l']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

,user_id,isbn,book_rating,book_title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel


In [23]:
book_totalratingCount=(combined_book_rating.groupby(by=['book_title'])['book_rating'].
                  count().
                  reset_index().
                  rename(columns={ 'book_rating':'totalRatingCount'})
                  [['book_title','totalRatingCount']]
                     )
book_totalratingCount.head()

,book_title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [24]:
rating_with_totalRatingCount = combine_book_rating.merge(book_totalratingCount, left_on = 'book_title', right_on = 'book_title', how = 'left')
rating_with_totalRatingCount.head()

,user_id,isbn,book_rating,book_title,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,53
1,2313,034545104X,5,Flesh Tones: A Novel,53
2,6543,034545104X,0,Flesh Tones: A Novel,53
3,8680,034545104X,5,Flesh Tones: A Novel,53
4,10314,034545104X,9,Flesh Tones: A Novel,53


In [25]:
pd.set_option('display.float_format', lambda x:'%.3f' %x)
print(book_totalratingCount['totalRatingCount'].describe())

count   230220.000
mean         4.088
std         15.577
min          1.000
25%          1.000
50%          1.000
75%          3.000
max       2264.000
Name: totalRatingCount, dtype: float64


In [26]:
print(book_totalratingCount['totalRatingCount'].quantile(np.arange(.9, 1,.01))),

0.900    7.000
0.910    7.000
0.920    8.000
0.930    9.000
0.940   11.000
0.950   12.000
0.960   15.000
0.970   19.000
0.980   27.000
0.990   47.000
Name: totalRatingCount, dtype: float64


(None,)

In [27]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,user_id,isbn,book_rating,book_title,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,53
1,2313,034545104X,5,Flesh Tones: A Novel,53
2,6543,034545104X,0,Flesh Tones: A Novel,53
3,8680,034545104X,5,Flesh Tones: A Novel,53
4,10314,034545104X,9,Flesh Tones: A Novel,53


In [28]:
book.duplicated().sum()     

0

In [29]:
#us and canada city

combined = rating_popular_book.merge(user, left_on = 'user_id', right_on = 'user_id', how = 'left')
combined_df=combined[combined['location'].str.contains('usa|canada')]
combined_df=combined_df.drop('age',axis=1)
combined_df.head()

,user_id,isbn,book_rating,book_title,totalRatingCount,location
0,276725,034545104X,0,Flesh Tones: A Novel,53,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,53,"cincinnati, ohio, usa"
2,6543,034545104X,0,Flesh Tones: A Novel,53,"strafford, missouri, usa"
3,8680,034545104X,5,Flesh Tones: A Novel,53,"st. charles county, missouri, usa"
4,10314,034545104X,9,Flesh Tones: A Novel,53,"beaverton, oregon, usa"


In [30]:
#aggregate rating.

agg_rating=combined.groupby('book_title').agg(mean_rating=('book_rating','mean'), number_of_rating=('book_rating','count')).reset_index()
                                              

In [31]:
agg_ratings=agg_rating[agg_rating['number_of_rating']>100]
agg_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 794 entries, 2 to 2180
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   book_title        794 non-null    object 
 1   mean_rating       794 non-null    float64
 2   number_of_rating  794 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 24.8+ KB


In [32]:
agg_ratings.sort_values(by='number_of_rating',ascending=False).head()

,book_title,mean_rating,number_of_rating
2150,Wild Animus,1.032,2264
1772,The Lovely Bones: A Novel,4.457,1164
1600,The Da Vinci Code,4.680,828
46,A Painted House,3.200,766
1806,The Nanny Diaries: A Novel,3.460,759


In [33]:
sns.jointplot(x='mean_rating', y='number_of_rating',data=agg_ratings)

<IPython.core.display.Javascript object>

In [34]:
df=pd.merge(combined, agg_ratings[['book_title']],on='book_title',how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155953 entries, 0 to 155952
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           155953 non-null  int64  
 1   isbn              155953 non-null  object 
 2   book_rating       155953 non-null  int64  
 3   book_title        155953 non-null  object 
 4   totalRatingCount  155953 non-null  int64  
 5   location          155953 non-null  object 
 6   age               155953 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 9.5+ MB


In [35]:
df

,user_id,isbn,book_rating,book_title,totalRatingCount,location,age
0,276727,446520802,0,The Notebook,580,"h, new south wales, australia",16.000
1,278418,446520802,0,The Notebook,580,"omaha, nebraska, usa",0.000
2,638,446520802,0,The Notebook,580,"san diego, california, usa",20.000
3,3363,446520802,0,The Notebook,580,"knoxville, tennessee, usa",29.000
4,7158,446520802,10,The Notebook,580,"omaha, nebraska, usa",30.000
...,...,...,...,...,...,...,...
155948,244635,425172996,0,Vector,126,"frederick, maryland, usa",31.000
155949,245215,425172996,5,Vector,126,"college station, texas, usa",44.000
155950,245410,425172996,9,Vector,126,"san antonio, texas, usa",28.000
155951,247133,425172996,10,Vector,126,"san diego, california, usa",0.000


In [36]:
#number of users
print('rating dataset has',df['user_id'].nunique(),'unique users')

#number of books
print('rating dataset has',df['book_title'].nunique(),'unique books')

#number of ratings
print('rating dataset has',df['book_rating'].nunique(),'unique rating')

#list unique rating
print('unique rating are',sorted(df['book_rating'].unique()))



rating dataset has 33391 unique users
rating dataset has 794 unique books
rating dataset has 11 unique rating
unique rating are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# user- book matrix

In [37]:
combined_df.head()

,user_id,isbn,book_rating,book_title,totalRatingCount,location
0,276725,034545104X,0,Flesh Tones: A Novel,53,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,53,"cincinnati, ohio, usa"
2,6543,034545104X,0,Flesh Tones: A Novel,53,"strafford, missouri, usa"
3,8680,034545104X,5,Flesh Tones: A Novel,53,"st. charles county, missouri, usa"
4,10314,034545104X,9,Flesh Tones: A Novel,53,"beaverton, oregon, usa"


In [38]:
from scipy.sparse import csr_matrix

combined_df=combined_df.drop_duplicates(['user_id','book_title'])
matrix=combined_df.pivot_table(index='book_title',columns='user_id',values='book_rating').fillna(0)
matrix_df = csr_matrix(matrix.values)

matrix.head()

user_id,8,9,14,16,17,23,26,32,39,42,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
book_title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
16 Lighthouse Road,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1984,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1st to Die: A Novel,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2010: Odyssey Two,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


# Applying Cosine Similarity

# item similarity matrix using cosine similarity


In [39]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(matrix)
similarity_scores.shape

(2180, 2180)

In [40]:
def recommend(book_name):
    # index fetch
    index = np.where(matrix.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = book[book['book_title'] == matrix.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('book_title')['book_title'].values))
        item.extend(list(temp_df.drop_duplicates('book_title')['book_author'].values))
        item.extend(list(temp_df.drop_duplicates('book_title')['image_url_m'].values))
        
        data.append(item)
    
    return data

In [41]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['American Psycho (Vintage Contemporaries)',
  'Bret Easton Ellis',
  'http://images.amazon.com/images/P/0679735771.01.MZZZZZZZ.jpg'],
 ['Lying Awake',
  'Mark Salzman',
  'http://images.amazon.com/images/P/0375406328.01.MZZZZZZZ.jpg']]

In [42]:
item_similarity = cosine_similarity(matrix.T)


In [43]:
# Convert the similarity matrix into a DataFrame
item_similarity_df = pd.DataFrame(item_similarity, index=matrix.columns, columns=matrix.columns)


# Applying KNN Algorithm

In [44]:
knn_model=NearestNeighbors( metric ='cosine', algorithm='brute',n_neighbors=7, n_jobs=-1)
knn_model.fit(matrix_df)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=7)

In [45]:
print("Algorithm:", knn_model.algorithm)
print("Metric:", knn_model.metric)
print("Leaf size:", knn_model.leaf_size)
print("Number of neighbors:", knn_model.n_neighbors)
print("P:", knn_model.p)
print("Radius:", knn_model.radius)

Algorithm: brute
Metric: cosine
Leaf size: 30
Number of neighbors: 7
P: 2
Radius: 1.0


In [46]:
# Get top 10 nearest neighbors 
indices=knn_model.kneighbors(matrix.loc[['1984']], 10, return_distance=False)

# Print the recommended books
print("Recommended Books:")

for index, value in enumerate(matrix.index[indices][0]):
    print((index+1),". ",value)

Recommended Books:
1 .  1984
2 .  Animal Farm
3 .  Brave New World
4 .  American Psycho (Vintage Contemporaries)
5 .  Lying Awake
6 .  The Drawing of the Three (The Dark Tower, Book 2)
7 .  Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death
8 .  The Gunslinger (The Dark Tower, Book 1)
9 .  The Hitchhiker's Guide to the Galaxy
10 .  Oranges Are Not the Only Fruit


/var/folders/s_/gb05tr917kx1q7y_b9cg68z00000gn/T/ipykernel_1640/863924489.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  for index, value in enumerate(matrix.index[indices][0]):


In [47]:
combined_df.head()

,user_id,isbn,book_rating,book_title,totalRatingCount,location
0,276725,034545104X,0,Flesh Tones: A Novel,53,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,53,"cincinnati, ohio, usa"
2,6543,034545104X,0,Flesh Tones: A Novel,53,"strafford, missouri, usa"
3,8680,034545104X,5,Flesh Tones: A Novel,53,"st. charles county, missouri, usa"
4,10314,034545104X,9,Flesh Tones: A Novel,53,"beaverton, oregon, usa"


# Performance of the model

In [48]:

#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#from sklearn.model_selection import train_test_split
 

In [49]:
from sklearn.model_selection import train_test_split
n_user=combined_df.user_id.unique().shape[0]
n_book=combined_df.book_title.unique().shape[0]
train_data,test_data=train_test_split(combined_df,test_size=0.25,random_state=42)

In [50]:
# Instantiate your recommendation model
model = RecommendationModel()

# Train the model
model.fit(train_data)


NameError: name 'RecommendationModel' is not defined